In [1]:
!pip install picos numpy swiglpk ipykernel --quiet

In [2]:
import picos
import numpy as np

## Planificación de Producción

In [5]:
import picos
import numpy as np

# Datos
costo_diurno = 8000
costo_nocturno = 4500
demanda = [2000, 3000, 2000, 3000]  # D1, N1, D2, N2
costo_almacenaje = 1

# Variables
P = picos.Problem()

# Variables binarias: si se produce en cada turno
y_d = picos.BinaryVariable('y_d', 2)  # diurno (dia 1 y dia 2)
y_n = picos.BinaryVariable('y_n', 2)  # nocturno (noche 1 y noche 2)

# Variables de produccion (cuantas unidades se producen en cada turno)
x_d = picos.RealVariable('x_d', 2, lower=0)
x_n = picos.RealVariable('x_n', 2, lower=0)

# Variables de inventario (cuanto se almacena al final de cada periodo, 3 momentos)
s = picos.RealVariable('s', 3, lower=0)

# Objetivo: costos fijos de produccion + costos de almacenaje
P.set_objective('min',
    costo_diurno * picos.sum(y_d) +
    costo_nocturno * picos.sum(y_n) +
    costo_almacenaje * picos.sum(s)
)

# Restricciones de produccion solo si se activa el turno
tipo_M = 10000
for i in range(2):
    P.add_constraint(x_d[i] <= tipo_M * y_d[i])
    P.add_constraint(x_n[i] <= tipo_M * y_n[i])

# Balance de inventario
# Periodo 0: D1
P.add_constraint(x_d[0] - demanda[0] == s[0])
# Periodo 1: N1
P.add_constraint(x_n[0] + s[0] - demanda[1] == s[1])
# Periodo 2: D2
P.add_constraint(x_d[1] + s[1] - demanda[2] == s[2])
# Periodo 3: N2
P.add_constraint(x_n[1] + s[2] - demanda[3] == 0)

P.solve(solver='glpk')
print('Produccion diurna:\n', np.round(x_d.value, 2))
print('Produccion nocturna:\n', np.round(x_n.value, 2))
print('Turnos diurnos activados:\n', [int(round(y_d[i].value)) for i in range(2)])
print('Turnos nocturnos activados:\n', [int(round(y_n[i].value)) for i in range(2)])
print('Inventario final en cada periodo:\n', np.round(s.value, 2))
print('Costo total:', round(P.value, 2))

Produccion diurna:
 [[2000.]
 [   0.]]
Produccion nocturna:
 [[5000.]
 [3000.]]
Turnos diurnos activados:
 [1, 0]
Turnos nocturnos activados:
 [1, 1]
Inventario final en cada periodo:
 [[   0.]
 [2000.]
 [   0.]]
Costo total: 19000.0


## Planificación Eléctrica

In [3]:
# ---- Datos ----
demand = picos.Constant('demanda',  [80, 100, 120, 140, 160])
limits  = picos.Constant('cap',     [70, 50, 60, 40])
c       = picos.Constant('c',       [20, 16, 18, 14])
o       = picos.Constant('o',       [1.5, 0.8, 1.3, 0.6])

# ---- Variables ----
T, P = 5, 4                      # anios, plantas
x = picos.RealVariable('x',  (P, T), lower=0)
alfa = picos.BinaryVariable('alfa', (P, T))
y = picos.BinaryVariable('y', P)

p = picos.Problem()

# ---- Restricciones ----
# (R1) Demanda anual
for j in range(T):
    p.add_constraint(picos.sum(x[:, j]) >= demand[j])

# (R2) Capacidad de planta
for i in range(P):
    p.add_constraint(picos.sum(x[i, :]) <= 1e4*y[i])

    for j in range(T):
        p.add_constraint(x[i, j] <= 1e4*alfa[i, j])
        p.add_constraint(x[i, j] - limits[i] <= 1e4*(1-alfa[i, j]))

# (R3) Permanencia
for i in range(P):
    for j in range(T-1):
        p.add_constraint(alfa[i, j+1] >= alfa[i, j])

# (R4) Operacion implica construccion
for i in range(P):
    for j in range(T):
        p.add_constraint(alfa[i, j] <= y[i])

# ---- Objetivo ----
p.set_objective('min', c[0]*y[0] + c[1]*y[1] + c[2]*y[2] + c[3]*y[3] +
                     o[0]*picos.sum(alfa[0, :]) + o[1]*picos.sum(alfa[1, :]) +
                     o[2]*picos.sum(alfa[2, :]) + o[3]*picos.sum(alfa[3, :]))

# ---- Solve ----
p.options.verbosity = 0
p.solve(solver='glpk')

print(x)           # energia generada
print(p.value)     # costo total minimo

[ 7.00e+01  5.00e+01  7.00e+01  7.00e+01  7.00e+01]
[ 1.00e+01  5.00e+01  5.00e+01  5.00e+01  5.00e+01]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  2.00e+01  4.00e+01]
62.7
